# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'02-23-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'02-23-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-02-24 05:29:16,33.93911,67.709953,55646,2435,48967,4244.0,Afghanistan,142.944699,4.375876
1,NaN,NaN,NaN,Albania,2021-02-24 05:29:16,41.15330,20.168300,102306,1696,65403,35207.0,Albania,3555.007297,1.657772
2,NaN,NaN,NaN,Algeria,2021-02-24 05:29:16,28.03390,1.659600,112279,2967,77382,31930.0,Algeria,256.046361,2.642524
3,NaN,NaN,NaN,Andorra,2021-02-24 05:29:16,42.50630,1.521800,10739,110,10285,344.0,Andorra,13898.919304,1.024304
4,NaN,NaN,NaN,Angola,2021-02-24 05:29:16,-11.20270,17.873900,20584,500,19207,877.0,Angola,62.629563,2.429071


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55492,55514,55518,55540,55557,55575,55580,55604,55617,55646
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,93075,93850,94651,95726,96838,97909,99062,100246,101285,102306
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,110711,110894,111069,111247,111418,111600,111764,111917,112094,112279


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2427,2427,2428,2428,2430,2430,2430,2432,2433,2435
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1555,1567,1582,1600,1617,1636,1653,1666,1681,1696
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2939,2943,2945,2947,2950,2954,2958,2961,2964,2967


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48395,48433,48514,48626,48798,48803,48820,48834,48895,48967
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,56764,57736,58794,59684,60675,61605,62533,63329,64318,65403
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,75999,76168,76330,76489,76640,76797,76940,77076,77225,77382


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6023,6024,6038,6050,6071,6079,6092,6117,6121,6143
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,19105,19136,19176,19267,19324,19361,19392,19433,19461,19554
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2042,2044,2055,2053,2057,2061,2067,2070,2074,2084


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,84,84,84,84,84,85,85,85,85,84
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,252,252,253,253,254,262,262,262,262,263
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,48,48,48,49,49,49,50,50,50,50


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-02-24 05:29:16,32.539527,-86.644082,6143,84,0,6059.0,"Autauga, Alabama, US",10995.364155,1.36741
689,1075.0,Lamar,Alabama,US,2021-02-24 05:29:16,33.779950,-88.096680,1306,32,0,1274.0,"Lamar, Alabama, US",9460.340456,2.45023
690,1077.0,Lauderdale,Alabama,US,2021-02-24 05:29:16,34.901719,-87.656247,8541,209,0,8332.0,"Lauderdale, Alabama, US",9210.710781,2.44702


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,28259987,502594,0
India,11030176,156567,10726702
Brazil,10257875,248529,9189903


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,28259987,502594,0,27757393,2021-02-24 05:29:16,37.936303,-91.379001
India,11030176,156567,10726702,146907,2021-02-24 05:29:16,23.088275,81.806127
Brazil,10257875,248529,9189903,819443,2021-02-24 05:29:16,-12.669522,-48.480493
United Kingdom,4146756,121536,11453,4013767,2021-02-24 05:29:16,30.744028,-35.367255
Russia,4142126,82666,3697787,361673,2021-02-24 05:29:16,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3541732,49822,0
Texas,2617096,42521,0
Florida,1878533,30213,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3541732,49822,0,3491910,2021-02-24 05:29:16,37.843962,-120.728594
Texas,2617096,42521,0,2574575,2021-02-24 05:29:16,31.660643,-98.653069
Florida,1878533,30213,0,1848320,2021-02-24 05:29:16,28.940755,-82.700744
New York,1604892,47037,0,1557855,2021-02-24 05:29:16,42.544151,-75.474183
Illinois,1177241,22528,0,1154713,2021-02-24 05:29:16,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1183378,20057,0
Arizona,Maricopa,506662,8909,0
Illinois,Cook,470933,9291,0
Florida,Miami-Dade,404499,5338,0
Texas,Harris,343573,4884,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1183378,20057,0,1163321,2021-02-24 05:29:16,34.308284,-118.228241,6037.0
Arizona,Maricopa,506662,8909,0,497753,2021-02-24 05:29:16,33.348359,-112.491815,4013.0
Illinois,Cook,470933,9291,0,461642,2021-02-24 05:29:16,41.841448,-87.816588,17031.0
Florida,Miami-Dade,404499,5338,0,399161,2021-02-24 05:29:16,25.611236,-80.551706,12086.0
Texas,Harris,343573,4884,0,338689,2021-02-24 05:29:16,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,55604,100246,111917,10699,20519,598,2064334,170402,28930,445374,...,4127578,52815,79654,1,136068,2383,173635,2165,74503,35796
2021-02-22,55617,101285,112094,10712,20548,614,2069751,170506,28937,446644,...,4138237,53310,79681,1,136545,2392,174969,2176,75027,35862
2021-02-23,55646,102306,112279,10739,20584,636,2077228,170672,28939,448371,...,4146760,53973,79717,1,136986,2403,176377,2187,75582,35910


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,2432,1666,2961,107,499,13,51198,3164,909,8386,...,120810,574,622,0,1316,35,1976,619,1020,1436
2021-02-22,2433,1681,2964,109,499,14,51359,3167,909,8397,...,120988,583,622,0,1320,35,1986,619,1031,1441
2021-02-23,2435,1696,2967,110,500,14,51510,3171,909,8434,...,121536,588,622,0,1325,35,1994,620,1040,1448


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,48834,63329,77076,10206,19013,218,1866501,161994,22882,420509,...,11315,46265,78152,1,128088,1702,161410,1432,67944,32125
2021-02-22,48895,64318,77225,10245,19190,222,1872213,162098,22885,421811,...,11346,46845,78171,1,128579,1717,162025,1432,68928,32216
2021-02-23,48967,65403,77382,10285,19207,233,1877898,162329,22892,423014,...,11457,47512,78185,1,129027,1760,162757,1432,69436,32288


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6023,6024,6038,6050,6071,6079,6092,6117,6121,6143
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1281,1281,1283,1282,1286,1291,1292,1294,1296,1306
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,8471,8476,8479,8480,8508,8513,8515,8523,8525,8541
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2777,2780,2781,2781,2787,2790,2794,2795,2798,2806
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,14608,14622,14641,14684,14729,14766,14779,14798,14809,14827


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,486843,57393,0,807967,315514,3531846,420614,273101,84732,49,...,765137,2600660,366735,14493,2575,564115,333794,129616,612240,53795
2021-02-22,487520,57738,0,809474,315759,3536946,421294,275334,85090,49,...,766089,2606513,367073,14608,2579,565270,334962,129854,612712,53900
2021-02-23,488973,57793,0,810658,316593,3541732,422390,276691,85228,49,...,767315,2617096,367789,14691,2589,567039,335693,130139,613419,53944


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-02-21        6117   19433    2070  2416   6042    1151   1940   12940   
2021-02-22        6121   19461    2074  2417   6043    1153   1945   13017   
2021-02-23        6143   19554    2084  2432   6058    1160   1948   13063   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-21         3367     1753  ...    2597    580     2992      671   
2021-02-22         3367     1753  ...    2603    583     2997      671   
2021-02-23         3382     1757  ...    2603    583     2999      671   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-21           3660  3326  2027          0      877    619  
2021-02-22           3686  3331  2030          0      879    620  
2021-02-23           3699  3333  2036          0      880    621  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,9592,289,0,15505,5357,49345,5892,7523,1367,0,...,11133,42162,1852,197,25,7331,4822,2261,6871,662
2021-02-22,9592,290,0,15502,5363,49541,5893,7562,1368,0,...,11153,42301,1853,198,25,7486,4857,2263,6871,662
2021-02-23,9660,290,0,15650,5377,49822,5907,7572,1379,0,...,11198,42521,1865,199,25,7658,4881,2274,6910,671


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-02-21          85     262      50   58    125      33     64     278   
2021-02-22          85     262      50   58    125      33     64     278   
2021-02-23          84     263      50   59    125      33     65     281   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-02-21           99       37  ...      26     11       27        7   
2021-02-22           99       37  ...      26     11       27        7   
2021-02-23          102       37  ...      26     11       27        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-21             34     9    12          0       26      5  
2021-02-22             34     9    12          0       26      5  
2021-02-23             34     9    12          0       26      5  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000432,0.011952,0.001369,0.002530,0.000976,0.000000,0.001800,0.000987,0.000138,0.004144,...,0.002388,0.012499,0.000276,0.0,0.003429,0.006334,0.007660,0.003709,0.008242,0.000783
2021-02-22,0.000234,0.010365,0.001582,0.001215,0.001413,0.026756,0.002624,0.000610,0.000242,0.002852,...,0.002582,0.009372,0.000339,0.0,0.003506,0.003777,0.007683,0.005081,0.007033,0.001844
2021-02-23,0.000521,0.010080,0.001650,0.002521,0.001752,0.035831,0.003613,0.000974,0.000069,0.003867,...,0.002060,0.012437,0.000452,0.0,0.003230,0.004599,0.008047,0.005055,0.007397,0.001338


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000823,0.007864,0.001014,0.000000,0.002008,0.181818,0.001487,0.001583,0.0,0.002151,...,0.001799,0.008787,0.0,NaN,0.003049,0.0,0.002537,0.001618,0.003937,0.002793
2021-02-22,0.000411,0.009004,0.001013,0.018692,0.000000,0.076923,0.003145,0.000948,0.0,0.001312,...,0.001473,0.015679,0.0,NaN,0.003040,0.0,0.005061,0.000000,0.010784,0.003482
2021-02-23,0.000822,0.008923,0.001012,0.009174,0.002004,0.000000,0.002940,0.001263,0.0,0.004406,...,0.004529,0.008576,0.0,NaN,0.003788,0.0,0.004028,0.001616,0.008729,0.004858


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000287,0.012729,0.001768,0.003540,0.000421,0.004608,0.002723,0.000562,0.000044,0.002924,...,0.001593,0.008369,0.000320,0.0,0.003840,0.046097,0.004025,0.0,0.014953,0.000904
2021-02-22,0.001249,0.015617,0.001933,0.003821,0.009309,0.018349,0.003060,0.000642,0.000131,0.003096,...,0.002740,0.012536,0.000243,0.0,0.003833,0.008813,0.003810,0.0,0.014483,0.002833
2021-02-23,0.001473,0.016869,0.002033,0.003904,0.000886,0.049550,0.003037,0.001425,0.000306,0.002852,...,0.009783,0.014238,0.000179,0.0,0.003484,0.025044,0.004518,0.0,0.007370,0.002235


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,0.001763,0.000000,NaN,0.002238,0.000901,0.001278,0.001910,0.000000,0.002378,0.0,...,0.001478,0.002639,0.001915,0.009332,0.000000,0.004099,0.000000,0.001948,0.000737,0.002086
2021-02-22,0.001391,0.006011,NaN,0.001865,0.000777,0.001444,0.001617,0.008176,0.004225,0.0,...,0.001244,0.002251,0.000922,0.007935,0.001553,0.002047,0.003499,0.001836,0.000771,0.001952
2021-02-23,0.002980,0.000953,NaN,0.001463,0.002641,0.001353,0.002602,0.004929,0.001622,0.0,...,0.001600,0.004060,0.001951,0.005682,0.003877,0.003129,0.002182,0.002195,0.001154,0.000816


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-21      0.004104  0.002114  0.001451  0.000829  0.000331  0.001741   
2021-02-22      0.000654  0.001441  0.001932  0.000414  0.000166  0.001738   
2021-02-23      0.003594  0.004779  0.004822  0.006206  0.002482  0.006071   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-21      0.001032  0.001936  0.000892  0.000000  ...  0.002703   
2021-02-22      0.002577  0.005951  0.000000  0.000000  ...  0.002310   
2021-02-23      0.001542  0.003534  0.004455  0.002282  ...  0.000000   

Province_State                                                              \
Admin2            Platte  Sheridan Sublette Sweetwater     Teton     Uinta   
2021-02-21      0.000000  0.001339      0.0   0.004115  0.002411  0.001482   
2021-02-22      0.005172  0.001671      0.0   0.007104  0.001503  0.001480   
2021-02-23      0.000000  0.000667      0.0   0.003527  0.000600  0.002956   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-21            NaN  0.000000  0.000000  
2021-02-22            NaN  0.002281  0.001616  
2021-02-23            NaN  0.001138  0.001613  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,0.000209,0.00000,NaN,0.001615,0.001683,0.004581,0.000849,0.000000,0.000732,NaN,...,0.001619,0.002878,0.005429,0.005102,0.0,0.018619,0.000000,0.003106,0.000000,0.000000
2021-02-22,0.000000,0.00346,NaN,-0.000193,0.001120,0.003972,0.000170,0.005184,0.000732,NaN,...,0.001796,0.003297,0.000540,0.005076,0.0,0.021143,0.007258,0.000885,0.000000,0.000000
2021-02-23,0.007089,0.00000,NaN,0.009547,0.002610,0.005672,0.002376,0.001322,0.008041,NaN,...,0.004035,0.005201,0.006476,0.005051,0.0,0.022976,0.004941,0.004861,0.005676,0.013595


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin Barbour      Bibb Blount Bullock    Butler   
2021-02-21      0.000000  0.000000     0.0  0.000000    0.0     0.0  0.000000   
2021-02-22      0.000000  0.000000     0.0  0.000000    0.0     0.0  0.000000   
2021-02-23     -0.011765  0.003817     0.0  0.017241    0.0     0.0  0.015625   

Province_State                               ... Wyoming                  \
Admin2           Calhoun  Chambers Cherokee  ...    Park Platte Sheridan   
2021-02-21      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2021-02-22      0.000000  0.000000      0.0  ...     0.0    0.0      0.0   
2021-02-23      0.010791  0.030303      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-02-21          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-02-22          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-02-23          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000318,0.011677,0.001455,0.002634,0.001106,0.031836,0.002498,0.001106,0.000149,0.004154,...,0.002567,0.012867,0.000388,0.0,0.003324,0.005886,0.005704,0.002199,0.009145,0.001762
2021-02-22,0.000276,0.011021,0.001518,0.001924,0.001259,0.029296,0.002561,0.000858,0.000196,0.003503,...,0.002575,0.011119,0.000363,0.0,0.003415,0.004831,0.006693,0.003640,0.008089,0.001803
2021-02-23,0.000399,0.010551,0.001584,0.002222,0.001506,0.032563,0.003087,0.000916,0.000132,0.003685,...,0.002317,0.011778,0.000408,0.0,0.003322,0.004715,0.007370,0.004348,0.007743,0.001571


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000475,0.009292,0.001130,0.000019,0.001454,0.119429,0.002168,0.001054,7.641954e-21,0.002600,...,0.002972,0.009435,7.862838e-07,NaN,0.003339,6.643538e-54,0.003302,0.000841,0.008056,0.002941
2021-02-22,0.000443,0.009148,0.001072,0.009355,0.000727,0.098176,0.002656,0.001001,3.820977e-21,0.001956,...,0.002223,0.012557,3.931419e-07,NaN,0.003189,3.321769e-54,0.004181,0.000421,0.009420,0.003211
2021-02-23,0.000632,0.009036,0.001042,0.009265,0.001365,0.049088,0.002798,0.001132,1.910488e-21,0.003181,...,0.003376,0.010567,1.965710e-07,NaN,0.003488,1.660884e-54,0.004105,0.001018,0.009075,0.004035


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-02-21,0.000573,0.014087,0.001864,0.003439,0.000707,0.007588,0.002958,0.000718,0.000086,0.003190,...,0.003516,0.009414,0.000389,0.0,0.003584,0.026409,0.004110,0.000056,0.014279,0.003311
2021-02-22,0.000911,0.014852,0.001898,0.003630,0.005008,0.012968,0.003009,0.000680,0.000108,0.003143,...,0.003128,0.010975,0.000316,0.0,0.003709,0.017611,0.003960,0.000028,0.014381,0.003072
2021-02-23,0.001192,0.015861,0.001966,0.003767,0.002947,0.031259,0.003023,0.001052,0.000207,0.002998,...,0.006455,0.012607,0.000248,0.0,0.003597,0.021327,0.004239,0.000014,0.010875,0.002653


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,0.001756,0.000875,NaN,0.002250,0.001118,0.001625,0.002335,0.000864,0.003082,3.799825e-104,...,0.001560,0.002349,0.002114,0.008564,0.001255,0.003848,0.001426,0.002171,0.001054,0.001621
2021-02-22,0.001573,0.003443,NaN,0.002057,0.000947,0.001535,0.001976,0.004520,0.003653,1.899913e-104,...,0.001402,0.002300,0.001518,0.008249,0.001404,0.002948,0.002463,0.002004,0.000913,0.001787
2021-02-23,0.002277,0.002198,NaN,0.001760,0.001794,0.001444,0.002289,0.004724,0.002638,9.499563e-105,...,0.001501,0.003180,0.001734,0.006966,0.002641,0.003039,0.002323,0.002099,0.001033,0.001301


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-21      0.003095  0.002097  0.001893  0.001223  0.001450  0.001692   
2021-02-22      0.001874  0.001769  0.001913  0.000819  0.000808  0.001715   
2021-02-23      0.002734  0.003274  0.003367  0.003512  0.001645  0.003893   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-02-21      0.001790  0.002698  0.001545  0.000380  ...  0.001808   
2021-02-22      0.002184  0.004324  0.000773  0.000190  ...  0.002059   
2021-02-23      0.001863  0.003929  0.002614  0.001236  ...  0.001030   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-02-21      0.002335  0.000909  0.000206   0.003170  0.001909  0.001325   
2021-02-22      0.003754  0.001290  0.000103   0.005137  0.001706  0.001402   
2021-02-23      0.001877  0.000979  0.000051   0.004332  0.001153  0.002179   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-02-21           -1.0  0.000377  0.000253  
2021-02-22           -1.0  0.001329  0.000934  
2021-02-23           -1.0  0.001233  0.001274  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-02-21,0.003418,0.000827,NaN,0.004055,0.002063,0.006132,0.001465,0.000812,0.000476,NaN,...,0.002594,0.003651,0.005695,0.007227,0.000020,0.013313,0.001535,0.003392,0.001669,0.000367
2021-02-22,0.001709,0.002144,NaN,0.001931,0.001591,0.005052,0.000817,0.002998,0.000604,NaN,...,0.002195,0.003474,0.003118,0.006151,0.000010,0.017228,0.004397,0.002138,0.000834,0.000183
2021-02-23,0.004399,0.001072,NaN,0.005739,0.002101,0.005362,0.001596,0.002160,0.004322,NaN,...,0.003115,0.004337,0.004797,0.005601,0.000005,0.020102,0.004669,0.003500,0.003255,0.006889


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-02-21      0.001544  0.004313  0.005802  0.002252  0.004182  0.001009   
2021-02-22      0.000772  0.002156  0.002901  0.001126  0.002091  0.000505   
2021-02-23     -0.005496  0.002987  0.001450  0.009184  0.001046  0.000252   

Province_State                                              ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers      Cherokee  ...      Park   
2021-02-21      0.000008  0.006956  0.002662  3.420408e-06  ...  0.000636   
2021-02-22      0.000004  0.003478  0.001331  1.710204e-06  ...  0.000318   
2021-02-23      0.007815  0.007135  0.015817  8.551020e-07  ...  0.000159   

Province_State                                                                \
Admin2                Platte  Sheridan      Sublette    Sweetwater     Teton   
2021-02-21      7.515260e-10  0.002723  6.143234e-16  3.729437e-06  0.001961   
2021-02-22      3.757630e-10  0.001362  3.071617e-16  1.864719e-06  0.000981   
2021-02-23      1.878815e-10  0.000681  1.535809e-16  9.323593e-07  0.000490   

Province_State                                                       
Admin2                 Uinta Unassigned      Washakie        Weston  
2021-02-21      9.965865e-11  -0.501961  3.816895e-08  8.106232e-07  
2021-02-22      4.982932e-11  -0.501961  1.908447e-08  4.053116e-07  
2021-02-23      2.491466e-11  -0.501961  9.542237e-09  2.026558e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210223,AK,55560,NaN,NaN,NaN,totalTestsViral,1653425,38.0,1260.0,...,NaN,0,9,8e9b55832d194e6faf511a32f4d3362caac0960e,0,0,0,0,0,NaN
1,20210223,AL,488973,106067.0,1882180.0,NaN,totalTestsPeopleViral,2265086,762.0,45250.0,...,NaN,68,122,a802ef0164c769a63bb5f8bee8625c468e610c29,0,0,0,0,0,NaN
2,20210223,AR,316593,66327.0,2359571.0,NaN,totalTestsViral,2609837,545.0,14617.0,...,NaN,14,47,1afb529b3e7563e63c092ddf862fde42dab2cefd,0,0,0,0,0,NaN
3,20210223,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,dc0db6d8a59051100a40bd5f4bf3390fd5aa499b,0,0,0,0,0,NaN
4,20210223,AZ,810658,54503.0,2953210.0,NaN,totalTestsViral,7478323,1515.0,57072.0,...,NaN,148,78,fe772660e88b070e73f6400d39ad27ecec9e089a,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210223,AK,55560.0,NaN,NaN,NaN,totalTestsViral,1653425.0,38.0,1260.0,...,NaN,0,9,8e9b55832d194e6faf511a32f4d3362caac0960e,0,0,0,0,0,NaN
1,20210223,AL,488973.0,106067.0,1882180.0,NaN,totalTestsPeopleViral,2265086.0,762.0,45250.0,...,NaN,68,122,a802ef0164c769a63bb5f8bee8625c468e610c29,0,0,0,0,0,NaN
2,20210223,AR,316593.0,66327.0,2359571.0,NaN,totalTestsViral,2609837.0,545.0,14617.0,...,NaN,14,47,1afb529b3e7563e63c092ddf862fde42dab2cefd,0,0,0,0,0,NaN
3,20210223,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,dc0db6d8a59051100a40bd5f4bf3390fd5aa499b,0,0,0,0,0,NaN
4,20210223,AZ,810658.0,54503.0,2953210.0,NaN,totalTestsViral,7478323.0,1515.0,57072.0,...,NaN,148,78,fe772660e88b070e73f6400d39ad27ecec9e089a,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-23,AK,55560,NaN,NaN,NaN,totalTestsViral,1653425,38.0,1260.0,NaN,...,NaN,0,9,8e9b55832d194e6faf511a32f4d3362caac0960e,0,0,0,0,0,NaN
2021-02-23,AL,488973,106067.0,1882180.0,NaN,totalTestsPeopleViral,2265086,762.0,45250.0,NaN,...,NaN,68,122,a802ef0164c769a63bb5f8bee8625c468e610c29,0,0,0,0,0,NaN
2021-02-23,AR,316593,66327.0,2359571.0,NaN,totalTestsViral,2609837,545.0,14617.0,204.0,...,NaN,14,47,1afb529b3e7563e63c092ddf862fde42dab2cefd,0,0,0,0,0,NaN
2021-02-23,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,dc0db6d8a59051100a40bd5f4bf3390fd5aa499b,0,0,0,0,0,NaN
2021-02-23,AZ,810658,54503.0,2953210.0,NaN,totalTestsViral,7478323,1515.0,57072.0,447.0,...,NaN,148,78,fe772660e88b070e73f6400d39ad27ecec9e089a,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-02-23,AK,55560.0,NaN,NaN,NaN,totalTestsViral,1653425.0,38.0,1260.0,NaN,...,NaN,0,9,8e9b55832d194e6faf511a32f4d3362caac0960e,0,0,0,0,0,NaN
2021-02-23,AL,488973.0,106067.0,1882180.0,NaN,totalTestsPeopleViral,2265086.0,762.0,45250.0,NaN,...,NaN,68,122,a802ef0164c769a63bb5f8bee8625c468e610c29,0,0,0,0,0,NaN
2021-02-23,AR,316593.0,66327.0,2359571.0,NaN,totalTestsViral,2609837.0,545.0,14617.0,204.0,...,NaN,14,47,1afb529b3e7563e63c092ddf862fde42dab2cefd,0,0,0,0,0,NaN
2021-02-23,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,dc0db6d8a59051100a40bd5f4bf3390fd5aa499b,0,0,0,0,0,NaN
2021-02-23,AZ,810658.0,54503.0,2953210.0,NaN,totalTestsViral,7478323.0,1515.0,57072.0,447.0,...,NaN,148,78,fe772660e88b070e73f6400d39ad27ecec9e089a,0,0,0,0,0,NaN
